In [2]:
%cd ..

d:\HUST\20232\ML\Project_OCR\HandwritingRecognition


In [3]:
from model.MyCrnn import MyCRNN
from dataset import DatasetImg
from utils.utils import strLabelConverter

import os
import torch
from torch.utils.data import random_split
import tqdm

c:\Users\tandat17z\anaconda3\envs\env_tandat17z\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset = DatasetImg('data/4/img', 'data/4/label', 512, 32)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=64,
            shuffle=True)
test_dataloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=64,
            shuffle=True)

with open(os.path.join('data/mychar.txt'), 'r', encoding='utf-8') as f:
    alphabet = f.read().rstrip()
# print(alphabet)
converter = strLabelConverter(alphabet)
print(converter.numClass)
# --------------------- Create Model ---------------------------------
model = MyCRNN(converter.numClass, 100, 0.1).to('cpu')
# print(f"Model structure: {model}\n\n")
# for name, param in model.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

criterion = torch.nn.CTCLoss().to('cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

154


In [6]:
model

MyCRNN(
  (cnn): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pooling0): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pooling1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.

In [11]:
converter.encode(['hôm nay là thứ mấy.', 'hôm nay là thứ 2'])

(tensor([ 35,  76,  40,   1,  41,  28,  52,   1,  39,  64,   1,  47,  35, 128,
           1,  40,  94,  52,  13,   1,  35,  76,  40,   1,  41,  28,  52,   1,
          39,  64,   1,  47,  35, 128,   1,  17], dtype=torch.int32),
 tensor([19, 16], dtype=torch.int32))

In [ ]:
t = tqdm(iter(train_dataloader), total=len(train_dataloader), desc='Epoch {}'.format(1))
for batch_idx, (imgs, labels) in enumerate(t):
    targets, target_lenghts = converter.encode(labels)
    targets = targets.to('cpu')
    target_lenghts = target_lenghts.to('cpu')

    optimizer.zero_grad()

    preds = model(imgs)
    
    b, l, c = preds.shape
    preds_ = preds.permute(1, 0, 2).to('cpu')
    preds_lengths = torch.full(size=(b,), fill_value=l, dtype=torch.long).to('cpu')

    loss = criterion(preds_.log_softmax(2), targets, preds_lengths, target_lenghts) # ctc_loss chỉ dùng với cpu, dùng với gpu phức tạp hơn thì phải
    assert (not torch.isnan(loss) and not torch.isinf(loss)), "Loss value is NaN or Inf"

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    loss.backward()
    optimizer.step()

    # avg_loss += loss.detach().item()

    # _, enc_preds = preds.max(2)
    # sim_preds = self.converter.decode(enc_preds.view(-1), preds_lengths, raw = False)
    # avg_levenshtein_loss += self.converter.Levenshtein_loss(sim_preds, labels)

        return avg_loss/len(self.train_dataloader), avg_levenshtein_loss/self.train_dataloader.sampler.num_samples

In [8]:
imgs, labels = next(iter(train_dataloader))
imgs = imgs.to('cpu')

print(labels)
targets, target_lenghts = converter.encode(labels)
print(targets)
targets = targets.to('cpu')
target_lenghts = target_lenghts.to('cpu')

preds_ = model(imgs)

b, l, c = preds_.shape
preds = preds_.permute(1, 0, 2).to('cpu')
preds_lengths = torch.full(size=(b,), fill_value=l, dtype=torch.long).to('cpu')
loss = criterion(preds.log_softmax(2), targets, preds_lengths, target_lenghts) # ctc_loss chỉ dùng với cpu, dùng với gpu phức tạp hơn thì phải
loss  

('Chuc vo dich Bundesliga thu 3 lien tiep den vao nam 2001',)
tensor([30, 35, 48, 30,  1, 49, 42,  1, 31, 36, 30, 35,  1, 29, 48, 41, 31, 32,
        46, 39, 36, 34, 28,  1, 47, 35, 48,  1, 18,  1, 39, 36, 32, 41,  1, 47,
        36, 32, 43,  1, 31, 32, 41,  1, 49, 28, 42,  1, 41, 28, 40,  1, 17, 15,
        15, 16], dtype=torch.int32)


tensor(9.2001, grad_fn=<MeanBackward0>)

In [47]:
enc, len = converter.encode(labels)
enc, len

(tensor([ 41,  34,  91, 122,  36,   1,  85,  91, 125,  30,   1,  43,  35,  65,
          47,   1,  35,  28,  36,   1,  43,  35,  95,  41,   1,  30,  90,  40,
           1,  14,   1,  41,  34,  64,  52,  11,   1,  30,  93,   1,  46,  48,
          94,  47,   1,  47,  45,  91,  28,   1,  49,  64,   1,  30,  35,  36,
         108,  48,   1,  31, 117,  41,   1,  40, 120,  47,  13,   1,  30,  48,
         120,  30,   1,  46, 116,  41,  34,   1,  38,  69,  42],
        dtype=torch.int32),
 tensor([81], dtype=torch.int32))

In [22]:
import numpy as np
torch.set_printoptions(threshold=float('inf'))
np.set_printoptions(threshold=float('inf'))


In [51]:

input = torch.zeros(128, 1, 145) 
for i, _ in enumerate(enc):
    input[i, 0, _.data] = 100
input.shape

torch.Size([128, 1, 145])

In [54]:
criterion(input.log_softmax(2), 
          enc, 
          torch.asarray([128]),
          len)

tensor(2.8399)

In [15]:
_, preds_id  = preds_.max(2)
converter.decode(preds_id.view(-1), preds_lengths, raw=True)
            

"ãōd0ãðíèðèv1đù(ð òàùvũgèggnöösugzröìð:ö:gðơðmàeeô/-è-èqè:jjvhh((àààwm|ha_+íùhz\\ìđăiđ'}g-gcoùùùs\\%đ0wwáàà:àööù%ðleðg/ðpa8ðððeöeèj"

In [75]:
import Levenshtein

def cer_loss_one_image(sim_pred, label):
    loss = Levenshtein.distance(sim_pred, label) * 1.0 
    return loss

def cer_loss(sim_preds, labels):
    losses = []
    print(sim_preds.__len__())
    for i in range(sim_preds.__len__()):
        pred = sim_preds[i]
        text = labels[i]

        loss = cer_loss_one_image(pred, text)
        losses.append(loss)
    return losses

In [78]:
cer_loss(['hello0',], ['helllo',])

1


[2.0]